In [7]:
import sys
sys.path.append('../')
from primer_design.align_sequence import lines_to_dict,read,AlignmentF
import matplotlib.pyplot as plt
import re
import numpy as np
from collections import Counter,namedtuple
import xml.etree.ElementTree as ET

In [2]:
#path to input file.
inputFile = ''

In [13]:
record = 

In [12]:
record('af','atcg','2016','H1N1')

Record(accession='af', sequence='atcg', year='2016', strain='H1N1')

In [50]:
from dateutil.parser import parse

In [53]:
parse('1995').year == 1995

True

In [43]:
result = parse_accession_xml('/home/hui/Downloads/FluA_M_N.A._HxNx_accession.xml')

In [44]:
result['CQ840446']

{'accession': 'CQ840446',
 'length': '987',
 'host': 'Human',
 'segment': '7 (MP)',
 'serotype': 'H1N1',
 'country': 'USA',
 'region': 'N',
 'date': '1995',
 'name': 'Influenza A virus',
 'mutations': None,
 'vac_strain': None,
 'fulllength_plus': 'c'}

In [ ]:
>MT167220 A/Arizona/02/2020 2020/01/09 7 (MP)

In [3]:
class Record(namedtuple('Record',['accession','sequence','year','serotype'])):
    def __init__(self,accession,seq,year,strain):
        super().__init__(accession,seq.upper(),year,strain)
    
class FluRecords:
    def __init__(self,fastaFile,accessionFile):
        
        fasta = lines_to_dict(read(file))
        year_ptn = re.compile(r'((19|20)\d\d)/\d{0,2}/\d{0,2}')
        self.years = {} 
        for i in self.seqs:
            match = year_ptn.search(i)
            if match:
                year = int(match.groups()[0])
                self.years[i] = year
                if year<1920 or year>2020:
                    print(i)
            else:
                self.years[i] = None
    def parse_accession_xml(self,file):
        tree = ET.parse(file)
        root = tree.getroot()
        result = {}
        for item in root:
            entry = {}
            for data in item:
                entry[data.tag] = data.text
            result[entry['accession']] = entry
        return result
                
    def align(self):
        self.aln = AlignmentF(sequence=list(self.seqs.values()))
                
    def plot_year(self):
        year_counter = list(Counter([i for i in self.years.values() if i]).items())
        year_counter.sort()
        labels = [i[0] for i in year_counter]
        year_height =  [i[1] for i in year_counter]
        x_position = list(range(len(labels)))

        fig,ax = plt.subplots()
        ax.bar(x_position,year_height,)
        ax.set_xticks(x_position[::10])
        ax.set_xticklabels(labels[::10], )
        ax.set_title('Count in years')
        plt.show()
        return fig
    
    def fast_after_year(self,year):
        return {i:seq for i,seq in self.seqs.items() if self.years[i] and self.years[i]>=year}
    
    def write_fasta(self,year,filename):
        fasta = self.fast_after_year(year)
        with open(filename,'wt') as f:
            for name, seq in fasta.items():
                f.write(name+'\n')
                f.write(seq+'\n')
    def plot_logo(self,ax):
        return self.aln.dna_logo(save=False,show=False,ax=ax)
    
    def plot_info(self,ax):
        req = np.array(self.aln._freq())
        en = 2.88539008/max(sum(self.aln.count),1.5)
        info = (np.log(5)/np.log(2)-en-self.aln.entropy())
        return ax.plot(info)

In [4]:
files="""/home/hui/AptitudeUsers/R&D/Users/Hui Kang/flu/FluA_genes/FluA_HA_N.A._HxNx.fa
/home/hui/AptitudeUsers/R&D/Users/Hui Kang/flu/FluA_genes/FluA_NA_N.A._HxNx.fa
/home/hui/AptitudeUsers/R&D/Users/Hui Kang/flu/FluA_genes/FluA_NP_N.A._HxNx.fa
/home/hui/AptitudeUsers/R&D/Users/Hui Kang/flu/FluA_genes/FluA_NS_N.A._HxNx.fa
/home/hui/AptitudeUsers/R&D/Users/Hui Kang/flu/FluA_genes/FluA_PA_N.A._HxNx.fa
/home/hui/AptitudeUsers/R&D/Users/Hui Kang/flu/FluA_genes/FluA_PB1_NA_HxNx.fa
/home/hui/AptitudeUsers/R&D/Users/Hui Kang/flu/FluA_genes/FluA_PB2_NA_HxNx.fa
/home/hui/AptitudeUsers/R&D/Users/Hui Kang/flu/FluA_genes/InfluenzaA_M1_M2.fa""".split('\n')

In [ ]:
#load fasta then plot year then cut after 'postYear'
postYear = 2015
genename = 'PB2'
fasta = FastA(files[6])

fig = fasta.plot_year()
# fig.savefig(f'./fluA_genes/fluA_{genename}_in_years.png')
fasta.write_fasta(postYear,f'./fluA_genes/fluA_{genename}_N.A._HxNx_{postYear}.fasta')

In [ ]:
# draw align from alignment sequences.
for gene in ['PB1','PB2']:
    alignFasta = FastA(f"./fluA_genes/fluA_{gene}_N.A._HxNx_2015_aln.fasta")
    alignFasta.align()
    # plot the aligned sequences
    fig,ax = plt.subplots(2,figsize=(int((len(alignFasta.aln))*0.2),6))
    alignFasta.plot_info(ax[0])
    ax[0].set_title(f'{gene}_post_2015 align.png')
    ax[0].set_xlabel('nucleotide position')
    alignFasta.plot_logo(ax[1])
    ax[1].set_title(f'{gene}_post_2015 Logo.png')
    ax[1].set_xlabel('nucleotide position')
    fig.set_tight_layout(True)
    fig.savefig(f'./fluA_genes/{gene}_post_2015 align.svg')


In [5]:
fluAM = FastA(files[-1])

>AY130766 A/Brevig Mission/1/1918 1918// 7 (MP)


In [ ]:
fluAM.seqs.keys()

In [21]:
for k in fluAM.seqs.keys():
    if 'MT167220' in k:
        print(k)
        print(fluAM.years[k])

>MT167220 A/Arizona/02/2020 2020/01/09 7 (MP)
2020


In [70]:
class T(dict):
    def __getattr__(self,key):
        return self[key]
    def __setattr__(self,key,value):
        self[key] = value